In [1]:
from pathlib import Path

from WSI_active_learning import main

CLASSES = ['D', 'M', 'N']
INITIAL_WSI_PER_CLASS = [2]
NUM_WSI_PER_GENERATION = [3] 
NUM_PATCHES_PER_WSI = [20]    

#PATCH_STRATEGY = ['passive', 'random','confidence', 'entropy', 'coreset', 'badge'] 
PATCH_STRATEGY = ['confidence'] 

WSI_PATCH_SCORE = ["random", "confidence", "entropy", "cdal", "class_conf"] 
#WSI_PATCH_SCORE = ['class_conf'] # 'entropy', 'confidence', 'class_conf', 'cdal' 

WSI_REP_METHOD = ["random", "average", "cdal", "kl_dis_x", "kl_rep", "kl_rank", "js_dis_x", "js_rep", "js_rank"]
#WSI_REP_METHOD = ['kl_rep'] 


MODEL_NAME = 'resnet' #'vgg','resnet'
NUM_RUNS = 1    # RUN THE CODE K TIMES


check = True 
if check == True : 
    for _ in range(NUM_RUNS):
        trial_number = 1
        base_dir = "log/test/trial_"

        while Path(f"{base_dir}{trial_number}").exists():
            trial_number += 1
        log_dir = f"{base_dir}{trial_number}"
        Path(log_dir).mkdir(exist_ok=False, parents=True)
        print(f"Log dir: {log_dir}")
        
        for wsi_rep_method in WSI_REP_METHOD : 
            for wsi_patch_score in WSI_PATCH_SCORE:
                for patch_strategy in PATCH_STRATEGY : 

                    if wsi_patch_score == "random" : 
                        if wsi_rep_method != "random" : continue 

                    elif wsi_patch_score in ["confidence", "entropy"] : 
                        if wsi_rep_method != 'average' : continue 

                    elif wsi_patch_score == "cdal" : 
                        if wsi_rep_method != 'cdal' : continue 
                    
                    else : 
                        if wsi_rep_method not in ["kl_dis_x", "kl_rep", "kl_rank", "js_dis_x", "js_rep", "js_rank"] : continue

                    for initial_wsi_per_class in INITIAL_WSI_PER_CLASS:  
                        logger_name = f"{wsi_patch_score}_{wsi_rep_method}_{patch_strategy}_{initial_wsi_per_class}WSI"
                        log_file = f"{log_dir}/test_{MODEL_NAME}_{logger_name}.txt"
                    
                        for num_wsi_per_generation in NUM_WSI_PER_GENERATION:

                            for num_patches_per_wsi in NUM_PATCHES_PER_WSI: 
                                print(f" {wsi_rep_method}, {wsi_patch_score}, {patch_strategy}, {initial_wsi_per_class} initial wsi per class, {num_wsi_per_generation} wsi per generation, {num_patches_per_wsi} patches per wsi")
                                with open(log_file, "a") as logger:
                                    logger.write("cycle, number_of_wsi,number_of_patches_per_wsi, train_patches, auroc_score, accuracy\n")
                                    main(
                                    patch_strategy=patch_strategy,
                                    wsi_patch_score = wsi_patch_score, 
                                    wsi_rep_method = wsi_rep_method,
                                    initial_wsi_per_class=initial_wsi_per_class,
                                    num_wsi_per_generation=num_wsi_per_generation,
                                    num_patches_per_wsi=num_patches_per_wsi,
                                    logger=logger,
                                    trial_number=trial_number,
                                    base_dir = log_dir) 

    

c:\Users\user\.conda\envs\active_learning\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Log dir: log/test/trial_11
random, passive, 2 initial wsi per class, 3 wsi per generation, 20 patches per wsi
# of unlabled_pool:  4695
Cycle: 1. Unlabeled WSI: 20. # of training WSI: 6, # of training patches: 300


[1/2] train loss: 1.622, train acc: 0.427, val loss: 0.000, val acc: 0.000. Time: 10.402158sec


[2/2] train loss: 1.517, train acc: 0.397, val loss: 0.000, val acc: 0.000. Time: 4.682976sec


Test - loss: 24.180, acc 0.340, auc: 0.668



...

[1/2] train loss: 1.670, train acc: 0.353, val loss: 0.000, val acc: 0.000. Time: 4.565457sec


[2/2] train loss: 1.386, train acc: 0.440, val loss: 0.000, val acc: 0.000. Time: 3.866530sec


Test - loss: 3.445, acc 0.393, auc: 0.685



calculate representative value for each wsi:   0%|                           | 0/20 [00:00<?, ?it/s]


TypeError: cannot unpack non-iterable NoneType object